<h3>Reading Data</h3>
<p>The following code reads the data and displays a sample</p>

In [ ]:
import pandas as pd
''' I commented this part as the data file was used to generate other files that 
are now available under "Data" folder.'''
# GeneName has more than one data type
#data = pd.read_csv("Normalized_RNAseq_Data/normalized_counts_idmatched.csv", dtype={'GeneName': str})
#data_T = data.T
#print("Unique gene names count: ",data_T.iloc[1].unique())



<h3>Basic Data Information</h3>
<p>Here's some basic statistics of the data</p>

In [ ]:
#data.describe()

In [ ]:
#data.info()

<h3>Working with Metadata File</h3>

In [ ]:
import pandas as pd
meta_data = pd.read_csv("../Data/metadata.csv")
meta_data

<h3>Filtering Out</h3>

In [ ]:
week_48_mask = meta_data["Week"] == 48
filtered_by_week_48 = meta_data[week_48_mask]
filtered_by_week_48 # displaying data

<h3>Outputting Sample Numbers</h3>

In [ ]:
flare_1_mask = filtered_by_week_48["Flare"] == 1
flare_3_mask = filtered_by_week_48["Flare"] == 3
flare_1_48 = filtered_by_week_48[flare_1_mask]
flare_3_48 = filtered_by_week_48[flare_3_mask]
flare_1_sample_numbers_48 = flare_1_48['sample']
flare_3_sample_numbers_48 = flare_3_48['sample']
print("Flare Samples:\n", flare_1_sample_numbers_48,'\n\n')
print("No Flare Samples:\n", flare_3_sample_numbers_48)

<h3>Filterng All</h3>

In [ ]:
week_24_mask = meta_data["Week"] == 24
filtered_by_week_24 = meta_data[week_24_mask]
filtered_by_week_24

In [ ]:
flare_1_mask = filtered_by_week_24["Flare"] == 1
flare_3_mask = filtered_by_week_24["Flare"] == 3
flare_1_24 = filtered_by_week_24[flare_1_mask]
flare_3_24 = filtered_by_week_24[flare_3_mask]
flare_1_sample_numbers_24 = flare_1_24['sample']
flare_3_sample_numbers_24 = flare_3_24['sample']


<h3>Back to Original File</h3>
This section will transpose and filter the results

In [ ]:
# Transposing and resetting the index of the rows
data_T = data.T.reset_index()
#dropping the Gene name as it has missing values
data_T = data_T.drop(1).reset_index()
# Display result
data_T


In [ ]:
# Dropping the column that was generated from transposing the data
data_T = data_T.drop(columns='level_0')
#locating Gene IDs to set them as column names
column_names = data_T.iloc[0]
#setting the column names to the extracted Gene IDs
data_T.columns = column_names
data_T.rename(columns={'Geneid': 'Sample ID'}, inplace=True)
#dropping the row with Gene IDs
data_T = data_T.drop(0)
#Displaying the dataset
data_T

In [ ]:
#filter sample by flare/no flare
# No flare samples 
wk_48_flare_1 = data_T[data_T['Sample ID'].isin(flare_1_sample_numbers_48)]
# Flare samples
wk_48_flare_3 = data_T[data_T['Sample ID'].isin(flare_3_sample_numbers_48)]


In [ ]:
# Dropping columns with constants
# Identify numerical columns

# Finding columns with constants
wk_48_flare_1_const_columns = wk_48_flare_1.iloc[:, 1:].columns[wk_48_flare_1.iloc[:, 1:].std() == 0]
wk_48_flare_3_const_columns  = wk_48_flare_1.iloc[:, 1:].columns[wk_48_flare_1.iloc[:, 1:].std() == 0]

# I commented the print statements. Uncomment to see the gene ids
#print("Constant columns in 'wk_48_flare_1':", wk_48_flare_1_const_columns)
#print("Constant columns in 'wk_48_flare_3':", len(wk_48_flare_3_const_columns))

# Droping columns and saving the dataframe
wk_48_flare_1_no_constants = wk_48_flare_1.drop(columns=wk_48_flare_1_const_columns)
wk_48_flare_3_no_constants = wk_48_flare_3.drop(columns=wk_48_flare_3_const_columns)


<h3>Important Notes</h3>
First code chunk only drops columns with constants based on the standard deviation. This means if there's any value that's different in the column, the column will be kept.<br>
Second code chunk drops columns with more than half the values < 5.<br>

Both can be easily updated by updating the numbers in the code chunks.<br>
A list of dropped Gene ID's can be found in each code block by referring to the variables 'columns_to_drop_flare_x' where x = 1 or 3


In [ ]:
# Dropping the ones with zeros
# Chnage the threshold to change the numbner of columns to be dropped
threshold = 0.40
zero_percentage = (wk_48_flare_1_no_constants == 0).mean()
columns_to_drop_flare_1 = zero_percentage[zero_percentage >= threshold].index
print("Number of columns with",threshold*100,"% or more missing data in flare 1 dataset: ", len(columns_to_drop_flare_1))

zero_percentage = (wk_48_flare_3_no_constants == 0).mean()
columns_to_drop_flare_3 = zero_percentage[zero_percentage >= threshold].index
print("Number of columns with",threshold*100,"% or more missing data in flare 3 dataset: ", len(columns_to_drop_flare_3))

# drop columns
wk_48_flare_1_no_constants = wk_48_flare_1_no_constants.drop(columns=columns_to_drop_flare_1)
wk_48_flare_3_no_constants = wk_48_flare_3_no_constants.drop(columns=columns_to_drop_flare_3)



In [ ]:
# Dropping the ones with zeros
# Chnage the threshold to change the numbner of columns to be dropped
threshold = 0.50
zero_percentage = (wk_48_flare_1_no_constants.iloc[:, 1:] < 5).mean()
columns_to_drop_flare_1 = zero_percentage[zero_percentage >= threshold].index
print("Number of columns with",threshold*100,"% or more missing data in flare 1 dataset: ", len(columns_to_drop_flare_1))

zero_percentage = (wk_48_flare_3_no_constants.iloc[:, 1:] < 5).mean()
columns_to_drop_flare_3 = zero_percentage[zero_percentage >= threshold].index
print("Number of columns with ",threshold*100,"% or more missing data in flare 3 dataset: ", len(columns_to_drop_flare_3))

# drop columns
wk_48_flare_1_no_constants = wk_48_flare_1_no_constants.drop(columns=columns_to_drop_flare_1)
wk_48_flare_3_no_constants = wk_48_flare_3_no_constants.drop(columns=columns_to_drop_flare_3)


In [ ]:
# Flare 1 samples for week 48
wk_48_flare_1_no_constants

In [ ]:
# Flare 3 samples for week 48
wk_48_flare_3_no_constants

In [ ]:
in_1_not_in_3 = []
in_3_not_in_1 = []

for name in wk_48_flare_1_no_constants.columns:
    if name not in wk_48_flare_3_no_constants:
        in_1_not_in_3.append(name)

for name in wk_48_flare_3_no_constants.columns:
    if name not in wk_48_flare_1_no_constants:
        in_3_not_in_1.append(name)
print("in_1_not_in_3: ",len(in_1_not_in_3), "Percentage: ", len(in_1_not_in_3)/len(wk_48_flare_3_no_constants.columns)*100,'%')
print("in_1_not_in_3: ",in_1_not_in_3)
print("in_3_not_in_1: ", len(in_3_not_in_1), "Percentage: ", len(in_3_not_in_1)/len(wk_48_flare_1_no_constants.columns)*100,'%')
print("in_3_not_in_1: ", in_3_not_in_1)

In [ ]:
import numpy as np
gene_ids = np.array(data['Geneid'])
gene_names = np.array(data['GeneName'])
names_in_1_not_in_3 = []
names_in_3_not_in_1 = []

# get gene names for available in 1 missing 3
for geneid in in_1_not_in_3:
    # find its index and then the id based on the index
    names_in_1_not_in_3.append(gene_names[np.where(gene_ids == geneid)[0][0]])

for geneid in in_3_not_in_1:
    # find its index and then the id based on the index
    names_in_3_not_in_1.append(gene_names[np.where(gene_ids == geneid)[0][0]])

print("names_in_1_not_in_3: ",names_in_1_not_in_3)
print("names_in_3_not_in_1: ",names_in_3_not_in_1)


max_len = max(len(names_in_1_not_in_3), len(names_in_3_not_in_1))
names_in_1_not_in_3.extend([None] * (max_len - len(names_in_1_not_in_3)))
names_in_3_not_in_1.extend([None] * (max_len - len(names_in_3_not_in_1)))


genes_dict = {
    "names_in_1_not_in_3": names_in_1_not_in_3,
    "names_in_3_not_in_1": names_in_3_not_in_1
}
df = pd.DataFrame(genes_dict)

# Export DataFrame to CSV
#df.to_csv('../Data/missing_names.csv', index=False)

<h3>Getting the data ready for pyTorch</h3>
Combining both flare 1 and flare 3 samples of week 48. Adding the Flare column by combining the dataframes based on the sample ID and then removing the unnecessary columns from the resulting dataframe.



In [ ]:
# Preparing the dataset for pytorch
filtered_by_week_48
combined_wk48 = pd.concat([wk_48_flare_1_no_constants,wk_48_flare_3_no_constants])

#combining datasets
df_combined = pd.merge(filtered_by_week_48, combined_wk48, right_on='Sample ID', left_on='sample')
df_combined = df_combined.drop(columns = ['PaxGeneTubeID', 'sample','Week','subject','Sample ID' ])
df_combined
#df_combined.to_csv('../Data/processed_data.csv', index=False)

<h3>Sorting IDs</h3>
here we will sort gene IDs based on significance to see if selecting a subset makes a difference in the learning process

<h4>P-value Based sort and extraction</h4>
gene ids will be sorted based on the p-value where smaller p-value means higher significance for the gene ID.

In [ ]:
#### gene_list = pd.read_csv("geneList.csv")
print("Original list:")
#display(gene_list.head(10))
p_sorted_list = gene_list.sort_values(by = "P_Flare")
p_value_dataset = p_sorted_list.reset_index()[["GeneID","GeneName","P_Flare"]]
print("Sorted by p-value:")
#display(p_value_dataset.head(10))

# Generating data file
p_value_dataset.to_csv('../Data/p_value_data.csv', index=False)
p_value_dataset.iloc[751]

<h4>Log2 FC Based sort and extraction</h4>
gene ids will be sorted based on the Log2 FC where higher value means higher significance for the gene ID.

In [ ]:
Log2FC_sorted_list = gene_list.sort_values(by = "Log2FC_Y_VS_N",ascending=False)
Log2FC_dataset = Log2FC_sorted_list.reset_index()[["GeneID","GeneName","Log2FC_Y_VS_N"]]

print("Sorted by log2FC:")
display(Log2FC_dataset.head(10))

# Generating data file
#Log2FC_dataset.to_csv('../Data/log2fc_data.csv', index=False)

In [ ]:
#Testing
Log2FC_dataset["GeneID"][0:3]
selected_IDs = Log2FC_dataset["GeneID"][0:3].tolist()
selected_IDs.append("Flare")
display(df_combined[selected_IDs])

# filtering function

def selected_sample(filtering_set, original_data, number_of_ids):
    selected_IDs = filtering_set["GeneID"][0:number_of_ids].tolist()
    selected_IDs.append("Flare")
    return original_data[selected_IDs]  

display(selected_sample(filtering_set = Log2FC_dataset, original_data = df_combined, number_of_ids = 3))

In [ ]:
import pandas as pd
df = pd.read_csv("../Data/processed_data.csv")
df


In [ ]:
import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu

# make a function

def t_test(df, threshold=0.05):
    # Define the flare categories
    group_1 = df[df['Flare'] == 1].drop('Flare', axis=1)  # All samples where flare = 1
    group_3 = df[df['Flare'] == 3].drop('Flare', axis=1)  # All samples where flare = 3

    # Dictionary to store p-values for each gene
    p_values = {}

    # Loop over each gene (column) and perform the statistical test
    for gene in group_1.columns:
        # Extract the gene expression data for each group
        group_1_data = group_1[gene]
        group_3_data = group_3[gene]
        
        # Perform a t-test (you can also use mannwhitneyu for non-parametric test)
        t_stat, p_val = ttest_ind(group_1_data, group_3_data, equal_var=False)  # Welch's t-test
        
        # Store the p-value
        p_values[gene] = p_val

    # Convert the results into a DataFrame for easier visualization
    p_values_df = pd.DataFrame(list(p_values.items()), columns=['Gene', 'P-value'])
    p_values_df = p_values_df.sort_values(by='P-value')
    # Display the p-values
    print(p_values_df)

    # Drop the ones below 0.05 and save the gene names in a list
    significant_genes = p_values_df[p_values_df['P-value'] < threshold]['Gene'].tolist()
    return significant_genes

print(t_test(df, threshold=0.05))

In [ ]:
# Drop the ones below 0.05 and save the gene names in a list
significant_genes = p_values_df[p_values_df['P-value'] <= 0.05]['Gene'].tolist()
len(significant_genes)



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
gene_list = pd.read_csv("../Data/geneList.csv")
gene_list

In [ ]:
significant_genes = gene_list[gene_list['P_Flare'] <= 0.002]['GeneID'].tolist()
len(significant_genes)



In [ ]:
patient_data = pd.read_csv("../Data/processed_data.csv")
patient_data

# From all data, drop any gene  that is not in the significant genes list keep flare column
significant_data = patient_data[significant_genes + ['Flare']]
significant_data

# randomly select 4 rows where flare = 3 and 4 rows where flare = 1

flare_1_data = significant_data[significant_data['Flare'] == 1].sample(4)
flare_3_data = significant_data[significant_data['Flare'] == 3].sample(4)

# Combine the two datasets
combined_data = pd.concat([flare_1_data, flare_3_data])




# Create a heatmap where genes are on the y-axis and flare is on the x-axis
heatmap_data = combined_data.set_index('Flare').T  # Transpose to have genes on y-axis
sns.heatmap(heatmap_data, cmap='hsv', fmt=".1f")
plt.xlabel("Flare")
plt.ylabel("Genes")
plt.title("Gene Expression Heatmap by Flare")
plt.show()

combined_data